In [ ]:
import requests
import pandas as pd
import itertools

def fetch_bybit_klines(symbol, interval, limit=20000):
    url = "https://api.bybit.com/v5/market/kline"
    params = {
        "category": "spot",
        "symbol": symbol,
        "interval": interval,  # "1", "5" (seconds)
        "limit": limit
    }
    response = requests.get(url, params=params).json()

    if not response.get("result") or not response["result"].get("list"):
        raise ValueError(f"No data returned for {symbol} {interval}s: {response}")

    data = response["result"]["list"]
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'])
    df['timestamp'] = pd.to_datetime(df['timestamp'].astype('int64'), unit='ms')
    return df[['timestamp', 'close']]

data_crypto_1s = pd.DataFrame()
data_crypto_5s = pd.DataFrame()

for crypto in itertools.product(['BTCUSDT', 'ETHUSDT', 'DOGEUSDT'], ['1', '5']):
    data = fetch_bybit_klines(crypto[0], crypto[1], limit=1000)
    if crypto[1] == '1':
        if data_crypto_1s.empty:
            data_crypto_1s = data
            data_crypto_1s = data_crypto_1s.rename(columns={'close' : f'{crypto[0]}_close'})
        else:
            data_crypto_1s = data_crypto_1s.merge(data, on='timestamp')
            data_crypto_1s = data_crypto_1s.rename(columns={'close' : f'{crypto[0]}_close'})
    if crypto[1] == '5':
        if data_crypto_5s.empty:
            data_crypto_5s = data
            data_crypto_5s = data_crypto_5s.rename(columns={'close' : f'{crypto[0]}_close'})
        else:
            data_crypto_5s = data_crypto_5s.merge(data, on='timestamp')
            data_crypto_5s = data_crypto_5s.rename(columns={'close' : f'{crypto[0]}_close'})

display(data_crypto_5s, data_crypto_1s)



,timestamp,BTCUSDT_close,ETHUSDT_close,DOGEUSDT_close
0,2025-05-16 22:10:00,103485,2556.8,0.22207
1,2025-05-16 22:05:00,103517.1,2556.96,0.2227
2,2025-05-16 22:00:00,103536.5,2560.03,0.22309
3,2025-05-16 21:55:00,103612.2,2567.39,0.22376
4,2025-05-16 21:50:00,103565.8,2564.89,0.22372
...,...,...,...,...
995,2025-05-13 11:15:00,103534.3,2486.7,0.22757
996,2025-05-13 11:10:00,103457,2480.01,0.22685
997,2025-05-13 11:05:00,103522.7,2483.09,0.2272
998,2025-05-13 11:00:00,103546.4,2484.71,0.22758


,timestamp,BTCUSDT_close,ETHUSDT_close,DOGEUSDT_close
0,2025-05-16 22:14:00,103485,2556.8,0.22207
1,2025-05-16 22:13:00,103457,2554.84,0.22175
2,2025-05-16 22:12:00,103505,2555.91,0.22204
3,2025-05-16 22:11:00,103481.9,2554.17,0.2223
4,2025-05-16 22:10:00,103462.7,2553.34,0.2222
...,...,...,...,...
995,2025-05-16 05:39:00,103864.8,2586.22,0.2263
996,2025-05-16 05:38:00,103845.9,2586.82,0.22633
997,2025-05-16 05:37:00,103883.7,2590.42,0.22663
998,2025-05-16 05:36:00,103950,2591.79,0.22705


In [21]:
import os
path = 'c:\\Users\\Maciek\\algoTrading\\TRADING_BOTS'
os.chdir(path)
# os.getcwd()
data_crypto_1s.to_csv('data_crypto_1s.csv')
data_crypto_5s.to_csv('data_crypto_5s.csv')